# Урок 3. Инструменты работы и визуализации ч.2
Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame.

("2023-11-20", "Electronics", 100, 12000),

("2023-11-21", "Electronics", 110, 13000),

("2023-11-22", "Electronics", 105, 12500),

("2023-11-20", "Clothing", 300, 15000),

("2023-11-21", "Clothing", 280, 14000),

("2023-11-22", "Clothing", 320, 16000),

("2023-11-20", "Books", 150, 9000),

("2023-11-21", "Books", 200, 12000),

("2023-11-22", "Books", 180, 10000)

Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.
Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день.

In [1]:
!pip install pyspark

In [2]:
# Импорт библиотек и модулей

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, avg, mean, round
from pyspark.sql.window import Window

In [3]:
# Проверка на существование запущенной spark-сессии

if 'spark' in locals():
  spark.stop()

In [4]:
# Создание сессии

spark = SparkSession.builder\
  .appName("SalesAnalysis")\
  .getOrCreate()

In [5]:
# Данные

data = [
    ("2023-11-20", "Electronics", 100, 12000),
    ("2023-11-21", "Electronics", 110, 13000),
    ("2023-11-22", "Electronics", 105, 12500),
    ("2023-11-20", "Clothing", 300, 15000),
    ("2023-11-21", "Clothing", 280, 14000),
    ("2023-11-22", "Clothing", 320, 16000),
    ("2023-11-20", "Books", 150, 9000),
    ("2023-11-21", "Books", 200, 12000),
    ("2023-11-22", "Books", 180, 10000)
]

In [6]:
# Определение схемы

columns = ["date", "category", "quantity", "revenue"]

In [7]:
# # Или явное ручное задание схемы (с допущением нулевых значений)

# from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType

# columns = StructType([
#     StructField("date", StringType, True),
#     StructField("category", StringType, True),
#     StructField("quantity", IntegerType, True),
#     StructField("revenue", IntegerType, True)
# ])


In [8]:
df = spark.createDataFrame(data, schema=columns)

# # В задании столбец "дата" имеет строковый формат, но можно трансформировать её в формат даты:

# df_temp = spark.createDataFrame(data, schema=columns)

# df = df_temp.withColumn("date", to_date(col("date"), yyyy-MM-dd))

In [9]:
df.show()

+----------+-----------+--------+-------+
|      date|   category|quantity|revenue|
+----------+-----------+--------+-------+
|2023-11-20|Electronics|     100|  12000|
|2023-11-21|Electronics|     110|  13000|
|2023-11-22|Electronics|     105|  12500|
|2023-11-20|   Clothing|     300|  15000|
|2023-11-21|   Clothing|     280|  14000|
|2023-11-22|   Clothing|     320|  16000|
|2023-11-20|      Books|     150|   9000|
|2023-11-21|      Books|     200|  12000|
|2023-11-22|      Books|     180|  10000|
+----------+-----------+--------+-------+



In [10]:
# Определение оконной спецификации

window_spec = Window.partitionBy("category")

In [11]:
# Использование оконной функции для расчета среднего значения выручки по категориям
# с округлением до 4х знаков после запятой

df_avg_rev = df.withColumn("avg_revenue", round(mean("revenue").over(window_spec), 4))

In [12]:
# Вывод окна рассчетов

df_avg_rev.show()

+----------+-----------+--------+-------+-----------+
|      date|   category|quantity|revenue|avg_revenue|
+----------+-----------+--------+-------+-----------+
|2023-11-20|      Books|     150|   9000| 10333.3333|
|2023-11-21|      Books|     200|  12000| 10333.3333|
|2023-11-22|      Books|     180|  10000| 10333.3333|
|2023-11-20|   Clothing|     300|  15000|    15000.0|
|2023-11-21|   Clothing|     280|  14000|    15000.0|
|2023-11-22|   Clothing|     320|  16000|    15000.0|
|2023-11-20|Electronics|     100|  12000|    12500.0|
|2023-11-21|Electronics|     110|  13000|    12500.0|
|2023-11-22|Electronics|     105|  12500|    12500.0|
+----------+-----------+--------+-------+-----------+



In [13]:
# Удаление дублирующих строк, оставляя уникальные комбинации category, avg_revenue
df_cat_rev = df_avg_rev.select("category", "avg_revenue").distinct()

df_cat_rev.show()

+-----------+-----------+
|   category|avg_revenue|
+-----------+-----------+
|      Books| 10333.3333|
|   Clothing|    15000.0|
|Electronics|    12500.0|
+-----------+-----------+



In [14]:
# Или можно объединить эти 2 строки с округлением до 2х знаков после запятой

df_cat_rev_2 = df.withColumn("avg_revenue", round(mean("revenue")\
  .over(window_spec), 2))\
  .select("category", "avg_revenue").distinct()

df_cat_rev_2.show()

+-----------+-----------+
|   category|avg_revenue|
+-----------+-----------+
|      Books|   10333.33|
|   Clothing|    15000.0|
|Electronics|    12500.0|
+-----------+-----------+



In [15]:
# Применение pivot для преобразования данных

pivot_df = df_avg_rev.groupBy("category").pivot("date").avg("revenue")

In [16]:
# Показать результат

pivot_df.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|Electronics|   12000.0|   13000.0|   12500.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|      Books|    9000.0|   12000.0|   10000.0|
+-----------+----------+----------+----------+



In [17]:
# Останавливаем Spark-сессию

spark.stop()